In [ ]:
import datetime
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import time                         #per ottenere la data corrente
from keras.preprocessing import image
from keras.models import load_model
import tensorflow as tf

from bs4 import BeautifulSoup
import requests
import json

from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

import oandapyV20.endpoints.orders as orders
import oandapyV20                                           # step 1
from oandapyV20 import API  
import oandapyV20.endpoints.pricing as pricing  
import oandapyV20.endpoints.trades as trades  
import configparser  


from keras.models import Sequential    #librerie per CNN
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.callbacks import Callback
from keras.layers.core import Dense, Activation
from keras import layers
from keras.optimizers import SGD

from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras import optimizers



from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense

from keras.callbacks import ModelCheckpoint #choose best model


import numpy as np
import pandas as pd
account_id = 'xxxxxxxx'
access_token = 'xxxxxx'
api = API(access_token=access_token)  

In [ ]:
import datetime
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import time                         #per ottenere la data corrente
from keras.preprocessing import image
from keras.models import load_model
import tensorflow as tf

from bs4 import BeautifulSoup
import requests
import json

from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

import oandapyV20.endpoints.orders as orders
import oandapyV20                                           # step 1
from oandapyV20 import API  
import oandapyV20.endpoints.pricing as pricing  
import oandapyV20.endpoints.trades as trades  
import configparser  


from keras.models import Sequential    #librerie per CNN
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.callbacks import Callback
from keras.layers.core import Dense, Activation
from keras import layers
from keras.optimizers import SGD

from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras import optimizers



from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense

from keras.callbacks import ModelCheckpoint #choose best model


import numpy as np
import pandas as pd




account_id = 'xxxxxxx'
access_token = 'xxxxxx'
black=True
api = API(access_token=access_token)  
pip=0.0001
now_invest=datetime.datetime.now()
now_dato=datetime.datetime.now()
prezzo=10
lunghezza_vet=30
tempo_minuti=20
tempo_dato=60
time_sleep=1
num=0
with tf.device('/cpu:0'):
    model=load_model("model_best_weights_best.h5")
    vect=[]
    while True:
            now = datetime.datetime.now()

            time.sleep(1)
            k=0
            while k!=1:
                    try: 
                        r = trades.TradesList(account_id)
                        t = api.request(r)

                        k=1
                    except: 
                        pass


            for yi in range(0,(len(t['trades']))):

                id=t['trades'][yi]['id']
                val=float(t['trades'][yi]['unrealizedPL'])
                if abs(val)>= prezzo:
                    k=0
                    while k!=1:
                        try: 

                            r=trades.TradeClose(account_id, tradeID=int(id))
                            rv = api.request(r)

                            k=1
                        except: pass

            if (now - now_dato).seconds>=tempo_dato:
                now_dato = datetime.datetime.now()
                #prendi dato
                k=0
                while k!=1:

                    try:
                        page_link = 'http://webrates.truefx.com/rates/connect.html?f=htmlD'
                        # this is the url that we've already determined is safe and legal to scrape from.
                        page_response = requests.get(page_link, timeout=1)
                        page_content = BeautifulSoup(page_response.content, "html.parser")
                        inizio=page_content.text.find('USDGBP/JPY')
                        pag2=page_content.text[inizio+10:]
                        val1=pag2[0:4]
                        price=float(val1+pag2[110:113])
                        k=1
                    except:
                        pass

                inizio=page_content.text.find('USDGBP/JPY')
                pag2=page_content.text[inizio+10:]
                val1=pag2[0:4]
                price=float(val1+pag2[110:113])
                bid=round(price-0.00005,5)
                ask=round(price+0.00005,5)
                vect.append(price)

                if len(vect)<lunghezza_vet:
                    pass
                else:
                    if len(vect)>=lunghezza_vet:
                        
                                    
                        vect=vect[-lunghezza_vet:]

                        if (now - now_invest).seconds/60>=tempo_minuti:
                            now_invest=datetime.datetime.now()
                            u=np.array(vect)
                            d=u.std(axis=0)
                            mu=u.mean(axis=0)
                            if black: plt.style.use('dark_background')
                            plt.ylim(u.min(axis=0)-0.001,u.max(axis=0)+0.0001)
                            plt.bar(0, u[0], 10,color = [(0.5,0.5,0.5)])
                            gk=10
                            plt.axis('off')
                            for ki in range(1,len(u)):
                                if u[ki]>mu+d*3.75:
                                     plt.bar(gk, u[ki], 10,color = [(0.98,0.98,0.98)])
                                elif mu+3.75*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.95,0.95,0.95)])
                                elif mu+3.5*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.92,0.92,0.92)])
                                elif mu+3.25*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.89,0.89,0.89)])
                                elif mu+3*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.86,0.86,0.86)])
                                elif mu+2.75*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.83,0.83,0.83)])
                                elif mu+2.5*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.80,0.80,0.80)])
                                elif mu+2.25*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.77,0.77,0.77)])
                                elif mu+2*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.74,0.74,0.74)])
                                elif mu+1.75*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.71,0.71,0.71)])
                                elif mu+1.5*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.68,0.68,0.68)])
                                elif mu+1.25*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.65,0.65,0.65)])
                                elif mu+1*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.62,0.62,0.62)])
                                elif mu+0.75*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.59,0.59,0.59)])
                                elif mu+0.5*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.56,0.56,0.56)])        
                                elif mu+0.25*d<u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.53,0.53,0.53)])

                                elif u[ki]<mu-3.75*d:
                                     plt.bar(gk, u[ki], 10,color = [(0.47,0.47,0.47)])

                                elif mu-3.75*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.44,0.44,0.44)])

                                elif mu-3.5*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.41,0.41,0.41)])

                                elif mu-3.25*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.38,0.38,0.38)])

                                elif mu-3*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.35,0.35,0.35)])

                                elif mu-2.75*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.32,0.32,0.32)])

                                elif mu-2.5*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.29,0.29,0.29)])

                                elif mu-2.25*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.26,0.26,0.26)])

                                elif mu-2*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.23,0.23,0.23)])


                                elif mu-1.75*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.20,0.2,0.2)])

                                elif mu-1.5*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.17,0.17,0.17)])

                                elif mu-1.25*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.14,0.14,0.14)])

                                elif mu-1*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.11,0.11,0.11)])

                                elif mu-0.75*d>u[ki]:
                                      plt.bar(gk, u[ki], 10,color = [(0.08,0.08,0.08)])

                                elif mu-0.5*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.05,0.05,0.05)])

                                elif mu-0.25*d>u[ki]:
                                     plt.bar(gk, u[ki], 10,color = [(0.02,0.02,0.02)])

                                else:
                                     plt.bar(gk, u[ki], 10,color = [(0.5,0.5,0.5)])
                                gk=gk+10


                                plt.savefig("result\prova.png",dpi=5)
                                test_image = image.load_img('result\prova.png', target_size = (32, 24),grayscale=True)
                                #il passo precedente non basta perchè in ingresso alla rete abbiamo un oggetto di dimensione 64x64x3 (immagine a colori)
                                #quindi usiamo il comando successivo per creare il vettore 3D
                                test_image = image.img_to_array(test_image)

                                test_image = np.expand_dims(test_image, axis = 0)/255

                                risultato=-1
                                if model.predict(test_image)[0][0]>0.5: 
                                    risultato =1
                                else:
                                    risultato =0
                            k=0
                            while k!=1:
                                try:
                                    if (risultato==1):
                                        mo= MarketOrderRequest(instrument="EUR_USD",
                                             units=30000)
                                        r=orders.OrderCreate(account_id, data=mo.data)
                                        rv = api.request(r) 
                                        k=1


                                    elif (risultato==0):
                                        mo= MarketOrderRequest(instrument="EUR_USD",
                                             units=-30000)
                                        r=orders.OrderCreate(account_id, data=mo.data)
                                        rv = api.request(r) 
                                        k=1
                                except:
                                    pass
                                
 








    

In [ ]:

import datetime
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import time                         #per ottenere la data corrente
from keras.preprocessing import image
from keras.models import load_model
import tensorflow as tf

from bs4 import BeautifulSoup
import requests
import json

from oandapyV20.contrib.requests import MarketOrderRequest
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

import oandapyV20.endpoints.orders as orders
import oandapyV20                                           # step 1
from oandapyV20 import API  
import oandapyV20.endpoints.pricing as pricing  
import oandapyV20.endpoints.trades as trades  
import configparser  


from keras.models import Sequential    #librerie per CNN
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.callbacks import Callback
from keras.layers.core import Dense, Activation
from keras import layers
from keras.optimizers import SGD

from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras import optimizers



from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense

from keras.callbacks import ModelCheckpoint #choose best model


import numpy as np
import pandas as pd




account_id = 'XXXXXXXX'
access_token = 'XXXXXX'
black=True
api = API(access_token=access_token)  
pip=0.0001
now_invest=datetime.datetime.now()
now_dato=datetime.datetime.now()
prezzo=10
lunghezza_vet=30
tempo_minuti=20
tempo_dato=60
time_sleep=1
num=0
with tf.device('/cpu:0'):
    model=load_model("model_best_weights_best.h5")
    vect_EURUSD=[]

    
    df=pd.DataFrame()
    valuta='EUR_USD'
    while True:
            now = datetime.datetime.now() # la data di ora

            time.sleep(1) #aspetta un minuto
            disinvesti(account_id='XXXXXXX',prezzo=10)

            if (now - now_dato).seconds>=tempo_dato:  #se sono passati 60 secondi
                now_dato = datetime.datetime.now()    #settiamo nuova data
            
                dati=prendi_dato()                   #prendiamo il dato

                vect_EURUSD.append(dati['EUR_USD']['price'])                    #lo mettiamo del vettore


                if len(vect_EURUSD)<lunghezza_vet:           #il vettore deve essere lungo almeno di 30 min, altrimenti non andiamo avanti
                    pass
                else:
                    if len(vect_EURUSD)>=lunghezza_vet:      #se maggiore di 30 min, prende solo gli ultimi 30
                        vect_EURUSD=vect_EURUSD[-lunghezza_vet:]

                        if (now - now_invest).seconds/60>=tempo_minuti: #se sono passati 10 min investe di nuovo
                            now_invest=datetime.datetime.now()   #settoamo la nuova data del investimento
                            u=np.array(vect_EURUSD)
                            
                            plot_grap(u,black=True,dpi=5,file='result\prova.png') #creiamo grafico e salviamolo




                            risultato=-1
                            risultato_prec=-1
                            #facciamo il predict
                            risultato=predict_nr(modelfile="model_best_weights_best.h5",file='result\prova.png',larg=64,alt=48,prob=0.5,grayscale=True)
                            try:
                                del df               #cancelliamo se già esiste
                            except:
                                pass
                            df=crea_df(account_id)   #creiamo il dataframe
                                
 
                            """ REGOLE INVESTIMENTO
                            try:
                                s=df.loc['instrument']=='EUR_USD'
                                if df[list(s[s].index)].loc['segno'].values =='up' and df[list(s[s].index)].loc['value'].values>0:
                                    risultato_prec=1
                                elif df[list(s[s].index)].loc['segno'].values =='down' and df[list(s[s].index)].loc['value'].values>0:
                                    risultato_prec=0
                                else:
                                    risultato_prec=-1
                            except:
                                risultato_prec=risultato
                            if risultato==risultato_prec:
                                risultato=risultato
                            else:
                                risultato=-1
                            """
                                
                                
                            investi(risultato=risultato,account_id=account_id, unita=30000, valuta="EUR_USD") #investo
                
                
                
                            """ DISINVESTO"""


                                
                                

Using TensorFlow backend.
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grays

C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
C:\Users\user\Anaconda3\envs\tf-gpu\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please

In [1]:
def plot_grap(u,black=False,dpi=5,file='prova.png'):

    d=u.std(axis=0)
    mu=u.mean(axis=0)
    if black: plt.style.use('dark_background')
    plt.ylim(u.min(axis=0)-0.001,u.max(axis=0)+0.0001)
    plt.bar(0, u[0], 10,color = [(0.5,0.5,0.5)])
    gk=10
    plt.axis('off')

    for ki in range(1,len(u)):
        if u[ki]>mu+d*3.75:
             plt.bar(gk, u[ki], 10,color = [(0.98,0.98,0.98)])
        elif mu+3.75*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.95,0.95,0.95)])
        elif mu+3.5*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.92,0.92,0.92)])
        elif mu+3.25*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.89,0.89,0.89)])
        elif mu+3*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.86,0.86,0.86)])
        elif mu+2.75*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.83,0.83,0.83)])
        elif mu+2.5*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.80,0.80,0.80)])
        elif mu+2.25*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.77,0.77,0.77)])
        elif mu+2*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.74,0.74,0.74)])
        elif mu+1.75*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.71,0.71,0.71)])
        elif mu+1.5*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.68,0.68,0.68)])
        elif mu+1.25*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.65,0.65,0.65)])
        elif mu+1*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.62,0.62,0.62)])
        elif mu+0.75*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.59,0.59,0.59)])
        elif mu+0.5*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.56,0.56,0.56)])        
        elif mu+0.25*d<u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.53,0.53,0.53)])

        elif u[ki]<mu-3.75*d:
             plt.bar(gk, u[ki], 10,color = [(0.47,0.47,0.47)])

        elif mu-3.75*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.44,0.44,0.44)])

        elif mu-3.5*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.41,0.41,0.41)])

        elif mu-3.25*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.38,0.38,0.38)])

        elif mu-3*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.35,0.35,0.35)])

        elif mu-2.75*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.32,0.32,0.32)])

        elif mu-2.5*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.29,0.29,0.29)])

        elif mu-2.25*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.26,0.26,0.26)])

        elif mu-2*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.23,0.23,0.23)])


        elif mu-1.75*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.20,0.2,0.2)])

        elif mu-1.5*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.17,0.17,0.17)])

        elif mu-1.25*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.14,0.14,0.14)])

        elif mu-1*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.11,0.11,0.11)])

        elif mu-0.75*d>u[ki]:
              plt.bar(gk, u[ki], 10,color = [(0.08,0.08,0.08)])

        elif mu-0.5*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.05,0.05,0.05)])

        elif mu-0.25*d>u[ki]:
             plt.bar(gk, u[ki], 10,color = [(0.02,0.02,0.02)])

        else:
             plt.bar(gk, u[ki], 10,color = [(0.5,0.5,0.5)])
        gk=gk+10


    plt.savefig(file,dpi=dpi)
    
    

    
    
    
def crea_df(account_id):
        k=0
        while k!=1:
            try:
                d={}
                r = trades.OpenTrades(account_id)
                t = api.request(r)
                k=1
            except: pass

        for i in range(0,len(t['trades'])):
            id=t['trades'][i]['id']
            instrument=t['trades'][i]['instrument']
            date=t['trades'][i]['openTime']
            date=t['trades'][i]['openTime']
            giorno=date[0:10]
            ora=date[11:19]
            units=abs(int(t['trades'][i]['currentUnits']))


            if int(t['trades'][i]['currentUnits'])<0:
                segno='down'
            else:
                segno='up'

            value=float(t['trades'][i]['unrealizedPL'])

            d[id]={'instrument' :instrument, 'giorno': giorno,  'ora' : ora, 'units':units, 'segno' : segno, 'value':value}

        df=pd.DataFrame(d)
        return df
    
    

        
def investi(risultato=-1,account_id='XXXXXXX', unita=30000, valuta="EUR_USD"):
    k=0
    while k!=1:
        try:
            if (risultato==1):
                mo= MarketOrderRequest(instrument=valuta,
                     units=unita)
                r=orders.OrderCreate(account_id, data=mo.data)
                rv = api.request(r) 
                k=1


            elif (risultato==0):
                mo= MarketOrderRequest(instrument=valuta,
                     units=-unita)
                r=orders.OrderCreate(account_id, data=mo.data)
                rv = api.request(r) 
                k=1
        except:
            pass

        
        

def prendi_dato(): 
    k=0
    while k!=1:
        try:
            d={}
            page_link = 'https://webrates.truefx.com/rates/connect.html?f=csv'
            # this is the url that we've already determined is safe and legal to scrape from.
            page_response = requests.get(page_link, timeout=1)
            page_content = BeautifulSoup(page_response.content, "html.parser")
            k=1
        except: pass

    t=page_content.text.split("\n")
    for vec in range(0,len(t)-3):
        k=0
        while k<1:
            K=t[vec].split(",")
            ask=float(K[2]+K[3])-0.00005
            bid=float(K[4]+K[5])+0.00005
            price=round((ask+bid)/2,4)
            d[K[0].replace("/","_")]={'price':price}
            k=1
    return d



def disinvesti(account_id='XXXXXXXX',prezzo=10):
            k=0
            while k!=1:
                    try: 
                        r = trades.TradesList(account_id)
                        t = api.request(r)

                        k=1
                    except: 
                        pass


            for yi in range(0,(len(t['trades']))):

                id=t['trades'][yi]['id']
                val=float(t['trades'][yi]['unrealizedPL'])
                if (val<0 and abs(val)>= prezzo) or val>prezzo:
                    k=0
                    while k!=1:
                        try: 

                            r=trades.TradeClose(account_id, tradeID=int(id))
                            rv = api.request(r)

                            k=1
                        except: pass
                        
def predict_nr(modelfile="model_best_weights_best.h5",file='result\prova.png',larg=32,alt=24,prob=0.5,grayscale=True):
                                model=load_model(modelfile)
                                test_image = image.load_img(file, target_size = (larg, alt),grayscale=grayscale)
                                #il passo precedente non basta perchè in ingresso alla rete abbiamo un oggetto di dimensione 64x64x3 (immagine a colori)
                                #quindi usiamo il comando successivo per creare il vettore 3D
                                test_image = image.img_to_array(test_image)

                                test_image = np.expand_dims(test_image, axis = 0)/255

                                risultato=-1
                                if model.predict(test_image)[0][0]>prob: 
                                    risultato =1
                                elif model.predict(test_image)[0][0]<prob:
                                    risultato =0
                                else:
                                    risultato=-1
                                return risultato
    

In [ ]:
abs(10)

In [ ]:
t

In [ ]:

for k in range(0,(len(t['trades']))):
    print(k)
    id=t['trades'][k]['id']
    val=float(t['trades'][k]['unrealizedPL'])
    print(str(val))


In [ ]:
t

In [ ]:
black=True
api = API(access_token=access_token)  
pip=0.0001
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import time                         #per ottenere la data corrente
from keras.preprocessing import image
from keras.models import load_model
import tensorflow as tf

from bs4 import BeautifulSoup
import requests



with tf.device('/cpu:0'):
    model=load_model("model_best_weights.h5")

    while True:

        vect=np.zeros(30)

        for vi in range(0,len(vect)):

            # Here, we're just importing both Beautiful Soup and the Requests library
            k=0
            while k!=1:

                try:
                    page_link = 'http://webrates.truefx.com/rates/connect.html?f=htmlD'
                    # this is the url that we've already determined is safe and legal to scrape from.
                    page_response = requests.get(page_link, timeout=1)
                    page_content = BeautifulSoup(page_response.content, "html.parser")
                    k=1
                except:
                    pass

            inizio=page_content.text.find('USDGBP/JPY')
            pag2=page_content.text[inizio+10:]
            val1=pag2[0:4]
            price=float(val1+pag2[110:113])
            bid=round(price-0.00005,5)
            ask=round(price+0.00005,5)

            vect[vi]=price
            time.sleep(60)





        u=vect
        d=u.std(axis=0)
        mu=u.mean(axis=0)
        if black: plt.style.use('dark_background')
        plt.ylim(u.min(axis=0)-0.001,u.max(axis=0)+0.0001)
        plt.bar(0, u[0], 10,color = [(0.5,0.5,0.5)])
        gk=10
        plt.axis('off')
        for ki in range(1,len(u)):
            if u[ki]>mu+d*3.75:
                 plt.bar(gk, u[ki], 10,color = [(0.98,0.98,0.98)])
            elif mu+3.75*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.95,0.95,0.95)])
            elif mu+3.5*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.92,0.92,0.92)])
            elif mu+3.25*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.89,0.89,0.89)])
            elif mu+3*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.86,0.86,0.86)])
            elif mu+2.75*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.83,0.83,0.83)])
            elif mu+2.5*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.80,0.80,0.80)])
            elif mu+2.25*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.77,0.77,0.77)])
            elif mu+2*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.74,0.74,0.74)])
            elif mu+1.75*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.71,0.71,0.71)])
            elif mu+1.5*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.68,0.68,0.68)])
            elif mu+1.25*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.65,0.65,0.65)])
            elif mu+1*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.62,0.62,0.62)])
            elif mu+0.75*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.59,0.59,0.59)])
            elif mu+0.5*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.56,0.56,0.56)])        
            elif mu+0.25*d<u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.53,0.53,0.53)])

            elif u[ki]<mu-3.75*d:
                 plt.bar(gk, u[ki], 10,color = [(0.47,0.47,0.47)])

            elif mu-3.75*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.44,0.44,0.44)])

            elif mu-3.5*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.41,0.41,0.41)])

            elif mu-3.25*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.38,0.38,0.38)])

            elif mu-3*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.35,0.35,0.35)])

            elif mu-2.75*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.32,0.32,0.32)])

            elif mu-2.5*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.29,0.29,0.29)])

            elif mu-2.25*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.26,0.26,0.26)])

            elif mu-2*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.23,0.23,0.23)])


            elif mu-1.75*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.20,0.2,0.2)])

            elif mu-1.5*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.17,0.17,0.17)])

            elif mu-1.25*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.14,0.14,0.14)])

            elif mu-1*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.11,0.11,0.11)])

            elif mu-0.75*d>u[ki]:
                  plt.bar(gk, u[ki], 10,color = [(0.08,0.08,0.08)])

            elif mu-0.5*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.05,0.05,0.05)])

            elif mu-0.25*d>u[ki]:
                 plt.bar(gk, u[ki], 10,color = [(0.02,0.02,0.02)])

            else:
                 plt.bar(gk, u[ki], 10,color = [(0.5,0.5,0.5)])
            gk=gk+10

        plt.savefig("result\prova.png",dpi=5)
        test_image = image.load_img('result\prova.png', target_size = (32, 24),grayscale=True)
        #il passo precedente non basta perchè in ingresso alla rete abbiamo un oggetto di dimensione 64x64x3 (immagine a colori)
        #quindi usiamo il comando successivo per creare il vettore 3D
        test_image = image.img_to_array(test_image)

        test_image = np.expand_dims(test_image, axis = 0)/255
        
        risultato=-1
        if model.predict(test_image)[0][0]>0.5: 
            risultato =1
        else:
            risultato =0
        k=0
        while k!=1:
            try:
                if (risultato==1):
                    mo= MarketOrderRequest(instrument="EUR_USD",
                         units=3000,
                         takeProfitOnFill=TakeProfitDetails(price=(bid+3*pip)).data,
                         stopLossOnFill=StopLossDetails(price=(bid-4.5*pip)).data)
                    k=1


                elif (risultato==0):
                    mo = MarketOrderRequest(instrument="EUR_USD",
                         units=-3000,
                         takeProfitOnFill=TakeProfitDetails(price=(ask-3*pip)).data,
                         stopLossOnFill=StopLossDetails(price=(ask+4.5*pip)).data)
                    k=1
            except:
                pass
        if (k==1):
            r=orders.OrderCreate(account_id, data=mo.data)
            rv = api.request(r)         
        
        





In [ ]:
mo = MarketOrderRequest(instrument="EUR_USD",
     units=1,
     takeProfitOnFill=TakeProfitDetails(price=bid+3*0.00001).data,
     stopLossOnFill=StopLossDetails(price=bid-3*0.00001).data
)

r=orders.OrderCreate(account_id, data=mo.data)
rv = api.request(r)     
    
    

In [ ]:
mo=mktOrder = MarketOrderRequest(instrument="EUR_USD",
         units=100,
         takeProfitOnFill=TakeProfitDetails(price=2).data,
         stopLossOnFill=StopLossDetails(price=2).data
    )
r=orders.OrderCreate(account_id, data=mo.data)
rv = api.request(r)

In [ ]:
mo = MarketOrderRequest(instrument="EUR_USD", units=10000)


r = orders.OrderCreate(accountID, data=mo.data)

rv = client.request(r)


In [ ]:
pip=0.0001

mktOrder = MarketOrderRequest(
    instrument="EUR_USD",
    units=-10000,
    takeProfitOnFill=TakeProfitDetails(price=1).data,
    stopLossOnFill=StopLossDetails(price=2).data)


# create the OrderCreate request
r = orders.OrderCreate(account_id, data=mktOrder.data)
try:
    # create the OrderCreate request
    rv = api.request(r)
except oandapyV20.exceptions.V20Error as err:
    print(r.status_code, err)
else:
    print(json.dumps(rv, indent=2))

In [ ]:
import h5py
f = h5py.File('model_best_weights_59-150000.h5', 'r')
print(list(f.keys()))
# will get a list of layer names which you can use as index
d1 = f['model_weights']
d2 = f['optimizer_weights']
# <HDF5 dataset "kernel:0": shape (128, 1), type "<f4">
d2[0]

In [ ]:
import h5py
f = h5py.File('model_best_weights_59-150000.h5', 'r')
print(list(f.keys()))
# will get a list of layer names which you can use as index
#d = f['dense']['dense_1']['kernel:0']
# <HDF5 dataset "kernel:0": shape (128, 1), type "<f4">


In [ ]:

    
risultato

In [ ]:
api = API(access_token=access_token)  
account_id = 'XXXXXX'
access_token = 'XXXXXX'
import numpy as np
import time                         #per ottenere la data corrente


params ={ "instruments": "EUR_USD"  }  






    while True:

        vect=np.zeros(30)

        for vi in range(0,len(vect)):
            r = pricing.PricingInfo(accountID=account_id, params=params)  
            rv = api.request(r)  
            bid=float(rv['prices'][0]['bids'][0]['price'])
            vect[vi]=bid
            time.sleep(60)
            
print(vect)


In [ ]:


for k in range(0,100):
    page_link = 'http://webrates.truefx.com/rates/connect.html?f=htmlD'
    # this is the url that we've already determined is safe and legal to scrape from.
    page_response = requests.get(page_link, timeout=1)
    page_content = BeautifulSoup(page_response.content, "html.parser")



    inizio=page_content.text.find('USDGBP/JPY')
    pag2=page_content.text[inizio+10:]
    val1=pag2[0:4]
    price=float(val1+pag2[110:113])
    bid=round(price-0.00005,5)
    ask=round(price+0.00005,5)


    print(bid)


In [ ]:
while True:
        k=0
        while k!=1:
                try: 
                    r = trades.TradesList(account_id)
                    t = api.request(r)

                    k=1
                except: 
                    pass


        for yi in range(0,(len(t['trades']))):

            id=t['trades'][yi]['id']
            val=float(t['trades'][yi]['unrealizedPL'])
            if abs(val)>= prezzo:
                k=0
                while k!=1:
                    try: 

                        r=trades.TradeClose(account_id, tradeID=int(id))
                        rv = api.request(r)

                        k=1
                    except:

                        pass
                """
                if val==999999:
                    if risultato==1: 
                        try:
                            plt.savefig(r"training_wrong\down\\"+ str(num) +".png",dpi=5)

                        except: 
                            pass
                    elif risultato==0:
                        try:
                            plt.savefig(r"training_wrong\up\\"+ str(num) +".png",dpi=5)
                        except:
                            pass
                num+=1
                """
